In [ ]:
# default_exp dataset.image_classification

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import sys
import argparse
import logging
from os.path import join, dirname, basename
from datetime import datetime
from mlcore.annotation.folder_category_adapter import read_annotations as folder_category_read_annotations
from mlcore.annotation.multi_category_adapter import read_annotations as multi_category_read_annotations
from mlcore.annotation.multi_category_adapter import write_annotations as multi_category_write_annotations
from mlcore.dataset.core import Dataset, configure_logging
from mlcore.dataset.type import DatasetType

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE = 'annotations.csv'
DEFAULT_SPLIT = 0.2
DATA_SET_FOLDER = 'datasets'
DATASET_TYPE = DatasetType.IMAGE_CLASSIFICATION

In [ ]:
# export

logger = logging.getLogger(__name__)

# Dataset for image classification

> Creates a dataset for image classification. Single and multi label classification is supported.

Creating a data-set for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The data-set is created based on an image-set.

## Image-Set

Image-sets are collected images to build a data-set from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[image_set_type]*/*[image_set_name]*

Inside the `[image_set_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the image-set contains

## Data-Set Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[data_set_type]*/*[data_set_name]*
where `[data_set_type]` is the same as the corresponding `[image_set_type]` and `[data_set_name]` is the same as the corresponding `[image_set_name]`.

Inside the `[data_set_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the data-set contains

## Create a classification data-set

Classification data-set can be created from a classification image-set. 
All images are validated, if they belong to one of the given categories. If categories with no images are found or images belong to a category not listed in `categories.txt`, the data-set can not be created.

In [ ]:
# export


class ImageClassificationDataset(Dataset):
    """
    Classification dataset.
    `name`: The name of the dataset.
    `base_path`: The dataset base-path.
    `imageset_path`: The imageset source path.
    `categories_path`: The path to the categories.txt file.
    `annotations_path`: The path to the annotations-file.
    """

    def __init__(self, name, base_path, imageset_path, categories_path, annotations_path=None, create_tfrecord=False):
        super().__init__(name, base_path, imageset_path, categories_path, DATASET_TYPE, create_tfrecord)
        self.annotations_path = annotations_path
        # if no annotation file exist, generate annotations based on the folder names
        if not annotations_path:
            self.annotations = folder_category_read_annotations(self.train_val_folder)
        else:
            self.annotations = multi_category_read_annotations(annotations_path, self.train_val_folder)

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)

        # write the split train annotations
        if annotations_train:
            annotations_target_path = join(self.train_folder, DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            multi_category_write_annotations(annotations_target_path, annotations_train)

        # write the split val annotations
        if annotations_val:
            annotations_target_path = join(self.val_folder, DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE)
            self.logger.info('Write annotations to {}'.format(annotations_target_path))
            multi_category_write_annotations(annotations_target_path, annotations_val)

        return annotations_train, annotations_val

## Build a data-set

To build a data-set from an image-set. Handles currently classification and segmentation image-sets taken from the image-set-type, which is the parent folder, the image-set folder is located in. 

In [ ]:
# export


def build_dataset(category_file_path, output, annotation_file_path=None, split=DEFAULT_SPLIT, seed=None, sample=0,
                  create_tfrecord=False, dataset_name=None):
    """
    Build the dataset for training, Validation and test
    `category_file_path`: the filename of the categories file
    `output`: the dataset base folder to build the dataset in
    `annotation_file_path`: the file path to the annotation file
    `split`: the size of the validation set as percentage
    `seed`: random seed to reproduce splits
    `sample`: the size of the sample set as percentage
    `create_tfrecord`: Also create .tfrecord files.
    `dataset_name`: the name of the dataset, if not set infer from the category file path
    """
    log_memory_handler = configure_logging()

    path = dirname(category_file_path)

    # try to infer the data-set name if not explicitly set
    if dataset_name is None:
        dataset_name = basename(path)

    logger.info('Build parameters:')
    logger.info(' '.join(sys.argv[1:]))
    logger.info('Build configuration:')
    logger.info('category_file_path: {}'.format(category_file_path))
    logger.info('annotation_file_path: {}'.format(annotation_file_path))
    logger.info('create_tfrecord: {}'.format(create_tfrecord))
    logger.info('split: {}'.format(split))
    logger.info('seed: {}'.format(seed))
    logger.info('sample: {}'.format(sample))
    logger.info('dataset_type: {}'.format(DATASET_TYPE))
    logger.info('output: {}'.format(output))
    logger.info('name: {}'.format(dataset_name))

    logger.info('Start build {} dataset {} at {}'.format(DATASET_TYPE, dataset_name, output))

    dataset = ImageClassificationDataset(dataset_name, output, path, category_file_path, annotation_file_path)

    # create the dataset folders
    logger.info("Start create the dataset folders at {}".format(dataset.base_path))
    dataset.create_folders()
    logger.info("Finished create the dataset folders at {}".format(dataset.base_path))

    # create the build log file
    log_file_name = datetime.now().strftime("build_%Y.%m.%d-%H.%M.%S.log")
    file_handler = logging.FileHandler(join(dataset.folder, log_file_name), encoding="utf-8")
    log_memory_handler.setTarget(file_handler)

    # build the dataset
    dataset.build(split, seed, sample)

    logger.info('Finished build {} dataset {} at {}'.format(DATASET_TYPE, dataset_name, output))

## Run from command line

To run the dataset builder from command line, use the following command:
`python -m mlcore.dataset.image_classification [parameters]`

The following parameters are supported:
- `[categories]`: The path to the categories file. (e.g.: *categories.txt*)
- `--annotation`: The path to the image-set annotation file, the dataset is build from. (e.g.: *imagesets/classification/car_damage/annotations.csv* for classification, *imagesets/segmentation/car_damage/via_region_data.json* for segmentation)
- `--split`: The percentage of the data which belongs to validation set, default to *0.2* (=20%)
- `--seed`: A random seed to reproduce splits, default to None
- `--sample`: The percentage of the data which will be copied as a sample set with in a separate folder with "_sample" suffix. If not set, no sample dataset will be created.
- `--tfrecord`: Also create .tfrecord files.
- `--output`: The path of the dataset folder, default to *../datasets*.
- `--name`: The name of the dataset, if not explicitly set try to infer from categories file path.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("categories",
                        help="The path to the imageset categories file.")
    parser.add_argument("--annotation",
                        help="The path to the imageset annotation file, the dataset is build from.",
                        default=None)
    parser.add_argument("--split",
                        help="Percentage of the data which belongs to validation set.",
                        type=float,
                        default=0.2)
    parser.add_argument("--seed",
                        help="A random seed to reproduce splits.",
                        type=int,
                        default=None)
    parser.add_argument("--sample",
                        help="Percentage of the data which will be copied as a sample set.",
                        type=float,
                        default=0)
    parser.add_argument("--tfrecord",
                        help="Also create .tfrecord files.",
                        action="store_true")
    parser.add_argument("--output",
                        help="The path of the dataset folder.",
                        default=DATA_SET_FOLDER)
    parser.add_argument("--name",
                        help="The name of the dataset, if not explicitly set try to infer from categories file path.",
                        default=None)
    args = parser.parse_args()

    build_dataset(args.categories, args.output, args.annotation, args.split, args.seed, args.sample, args.tfrecord,
                  args.name)
